### Questions

### Objectives
YWBAT 
* hide api permissions in your notebook
* access api and use it in data collection

### Outline
* build a twitter data collection tool
* store our tweets in a mongodb

In [ ]:
import json
import pandas as pd
import tweepy as tw

from pprint import pprint

from pymongo import MongoClient

In [ ]:
with open("/Users/rafael/.ssh/twitter_app00.json") as f:
    d = json.load(f)
d.keys()

In [ ]:
oauth = tw.OAuthHandler(consumer_key=d["consumer_key"], consumer_secret=d["consumer_secret"])
oauth.set_access_token(key=d["access_token"], secret=d["access_token_secret"])

In [ ]:
api = tw.API(auth_handler=oauth)

## What can I do with Twitter's API?
* udpate a status

In [ ]:
api.update_status("Teaching this awesome class, makes me soooooooooo happy... #blessedAF")

In [ ]:
last_tweet_id = api.me()._json['status']['id']

api.destroy_status(last_tweet_id)

In [ ]:
def update_statuses(api, status_list):
    for status in status_list:
        api.update_status(status)
    pass


def delete_recent_tweet(api):
    last_tweet_id = api.me()._json["status"]["id"]
    print("found id")
    print("deleting that ish")
    api.destroy_status(last_tweet_id)
    print("deleted...whew....")
    pass

In [ ]:
ces2020_tweets = list()

In [ ]:
# search stuff
results = api.search("#ces2020", tweet_mode='extended')
for r in results:
    ces2020_tweets.append(r._json)

In [ ]:
len(ces2020_tweets)

boooooo!!!! only 15 results? Dude, I want thousands....

In [ ]:
# how do I check my tweepy version?
tw.__version__

In [ ]:
class MyStreamListener(tw.StreamListener): 
# putting tw.StreamListener in here makes MyStreamListener have the same methods and attributes
    
    
    def on_status(self, status):
#         try:
#             print(status.extended_tweet["full_text"])
#         except:
#             print('---------not extended status---------')
#             print(status.text)
        print("inserted tweet into list")
        ces2020_tweets.append(status._json)
        pass
        
     
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

        # returning non-False reconnects the stream, with backoff.

In [ ]:
# Make a streamer object
listener = MyStreamListener()

# make a stream using our listener
stream = tw.Stream(auth=oauth, listener=listener)

# Connect to Mongo and Create 'Table' (collection) for got tweets

In [ ]:
# client = MongoClient(host='localhost', port=27017)
# tweets = client["tweets"]

In [ ]:
# got_collection = tweets["got_collection"]

In [ ]:
ces_keywords = ["#ces2020", "#CES2020", "#CES"]
stream.filter(track=ces_keywords, is_async=True)

In [ ]:
len(ces2020_tweets)

In [ ]:
client = MongoClient(host='localhost', port=27017)

In [4]:
tweets = client['tweets']

In [5]:
ces_tweets = tweets['ces_tweets']

In [6]:
ces_tweets_list = list(ces_tweets.find())

In [7]:
len(ces_tweets_list)

408

# let's parse a tweet for information that we want

In [8]:
first_tweet = ces_tweets_list[0]

In [49]:
my_dict_list = []
for first_tweet in ces_tweets_list:
    d = {}
    d["username"] = first_tweet['user']['name']
    d["userhandle"] = first_tweet['user']['screen_name']
    d["location"] = first_tweet['user']['location']
    d["hashtags"] = first_tweet['entities']['hashtags']
    
    try:
        d["text"] = first_tweet['full_text']
    except:
        d['text'] = first_tweet['text']
    d["mentions"] = first_tweet['entities']['user_mentions']
    d["created_at"] = first_tweet['created_at']
    d["device"] = 'android' if 'android' in first_tweet['source'].lower() else 'iphone'
    
    my_dict_list.append(d)

In [51]:
my_dict_list[:3]

[{'username': 'NICk VAL',
  'userhandle': 'NICK6808',
  'location': 'CANCUN',
  'hashtags': [{'text': 'CES2020', 'indices': [37, 45]}],
  'text': 'RT @TIME: TIME’s 25 best products of #CES2020, including:\n\n🎨 Acer’s ConceptD 7 Ezel, a laptop doubling as an artist’s easel\n💡 Moon Ultra, a…',
  'mentions': [{'screen_name': 'TIME',
    'name': 'TIME',
    'id': 14293310,
    'id_str': '14293310',
    'indices': [3, 8]}],
  'created_at': 'Thu Jan 09 20:24:25 +0000 2020',
  'device': 'android'},
 {'username': 'Cosmo Connected',
  'userhandle': 'CosmoConnected',
  'location': 'Paris, France',
  'hashtags': [{'text': 'ces2020', 'indices': [51, 59]},
   {'text': 'ces', 'indices': [71, 75]},
   {'text': 'brakelight', 'indices': [76, 87]},
   {'text': 'safety', 'indices': [88, 95]},
   {'text': 'safetyfirst', 'indices': [96, 108]}],
  'text': 'Live demo of the Cosmo Ride on Oxelo’s escooter at #ces2020\n@Decathlon #ces #brakelight #safety #safetyfirst 🛴 https://t.co/cFJaGVelQI',
  'mentions': [

In [52]:
df = pd.DataFrame(my_dict_list)
df.head()

,username,userhandle,location,hashtags,text,mentions,created_at,device
0,NICk VAL,NICK6808,CANCUN,"[{'text': 'CES2020', 'indices': [37, 45]}]","RT @TIME: TIME’s 25 best products of #CES2020,...","[{'screen_name': 'TIME', 'name': 'TIME', 'id':...",Thu Jan 09 20:24:25 +0000 2020,android
1,Cosmo Connected,CosmoConnected,"Paris, France","[{'text': 'ces2020', 'indices': [51, 59]}, {'t...",Live demo of the Cosmo Ride on Oxelo’s escoote...,"[{'screen_name': 'Decathlon', 'name': 'Decathl...",Thu Jan 09 20:24:24 +0000 2020,iphone
2,swissnex Boston,swissnexBoston,"Cambridge, MA","[{'text': 'SwissTech', 'indices': [37, 47]}, {...",RT @SGE: .@CES Las Vegas - the first #SwissTec...,"[{'screen_name': 'SGE', 'name': 'Switzerland G...",Thu Jan 09 20:24:24 +0000 2020,iphone
3,MrAndmisess,andmisess,"Maharashtra, India",[],RT @Samsung: Vehicle-to-Everything underpinned...,"[{'screen_name': 'Samsung', 'name': 'Samsung E...",Thu Jan 09 20:24:24 +0000 2020,android
4,SEDI,sedi_slim,The Netherlands,"[{'text': 'CES2020', 'indices': [95, 103]}]",RT @ValaAfshar: The world generates more than ...,"[{'screen_name': 'ValaAfshar', 'name': 'Vala A...",Thu Jan 09 20:24:23 +0000 2020,iphone


### What did you learn?

* pprint - Alex
* parse data from a json - Alisa
* how to kill a haunted jupyter notebook by restarting kernel - Andy
* using loops to repeat tasks  - Johnny
* try/except to avoid breaks - Andy

In [55]:
p = 3

try:
    p + "a"
except:
    try:
        p + [1]
    except:
        print('stop')
        print(p)

stop
3
